In [1]:
import sys

sys.path.append("..")
from typing import List

import requests
from bs4 import BeautifulSoup
from googlesearch import search
from pydantic import BaseModel, Field
from rich import print

from brain.sdk import BrainClient

In [2]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [3]:
from typing import List, Union

from pydantic import BaseModel, Field


class Tool(BaseModel):
    script: str = Field(..., description="The Python script to be executed. Ensure it handles errors and outputs structured results.")
    pip_install: List[str] = Field(..., description="The list of pip packages required to execute the script.")
    completed: Union[bool, str] = Field(..., description="Indicates if the task is successfully completed. True if we are able to generate script, otherwise a string explaining why the script cannot be generated.")

In [4]:
@brain_client.reasoner(schema=Tool)
def autonomous_tool(task: str):
    """
    Generates standalone, executable Python scripts with no external dependencies.
    """
    system_prompt = """You are a Python script generator focused on creating fully autonomous code. Your scripts must be completely self-contained:

DO NOT generate scripts that:
- Require external inputs (API keys, file paths, user prompts, etc.)
- Use mock/placeholder data
- Do not Simulate or Mock artificial data
- Need manual configuration
- Access local files or environment variables
- Connect to external services without built-in test data
- Depend on system-specific resources
- Use unreliable or niche packages

Instead:
- Use standard library when possible
- Include test data within the script
- Handle all edge cases and errors
- Use only widely-used PyPI packages if needed
- Generate any required data programmatically
- If we cannot do it without external dependencies, explain why in 'completed' field and guide the tool maker to say it cannot be done

IMPORTANT:
**Do not simulate data or use mock data.**
**Do not use methods that require secrets or API keys or other blocking algorithms in script**
**If we cannot write the script with the above constraints, Fail and explain why in the 'completed' field.**
**if local file reference is given use approrpiate input path in the script to read the file or save the file etc..**

If task requires unavoidable external dependencies or simulated or mock data, explain why in 'completed' field and make it a fail case."""

    user_prompt = f"""Generate a standalone Python script for: "{task}"

Return:
- script: Fully executable Python code
- pip_install: Required packages (empty list if none needed)
- completed: True if successful, or explanation why task cannot be autonomous"""

    return system_prompt, user_prompt
autonomous_tool_id=autonomous_tool.register()

In [5]:
import os
import shutil
import subprocess
import tempfile
import venv
from typing import Dict, Union
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn


def execute_script_with_temp_venv(script: str, pip_install: list) -> Dict[str, Union[str, bool]]:
    result = {
        "script": script,
        "pip_install": pip_install,
        "success": False,
        "stdout": "",
        "stderr": "",
        "error": ""
    }
    temp_dir = None

    # Create a Rich progress bar
    with Progress(
        SpinnerColumn(),
        TextColumn("[progress.description]{task.description}"),
        BarColumn(),
        TextColumn("{task.completed}/{task.total}"),
    ) as progress:
        try:
            # Step 1: Create a temporary directory for the virtual environment
            task = progress.add_task("Setting up environment...", total=3)
            temp_dir = tempfile.mkdtemp()
            venv_dir = os.path.join(temp_dir, "venv")
            venv.create(venv_dir, with_pip=True)
            progress.advance(task)

            # Step 2: Paths for the virtual environment
            venv_python = os.path.join(venv_dir, "bin", "python") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "python")
            venv_pip = os.path.join(venv_dir, "bin", "pip") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "pip")

            # Step 3: Install required packages
            progress.update(task, description="Installing dependencies...")
            if pip_install:
                install_command = [venv_pip, "install"] + pip_install
                subprocess.run(install_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            progress.advance(task)

            # Step 4: Write the script to a temporary file
            progress.update(task, description="Writing script to file...")
            script_path = os.path.join(temp_dir, "script.py")
            with open(script_path, "w") as script_file:
                script_file.write(script)

            # Step 5: Execute the script
            progress.update(task, description="Executing script...")
            execute_command = [venv_python, script_path]
            process = subprocess.run(
                execute_command, check=False, capture_output=True, text=True
            )
            progress.advance(task)

            # Step 6: Update result based on execution outcome
            result["stdout"] = process.stdout.strip()
            result["stderr"] = process.stderr.strip()
            result["error"] = ""
            result["success"] = process.returncode == 0 and not result["stderr"]

        except subprocess.CalledProcessError as e:
            result["error"] = f"Pip installation or execution failed: {str(e)}"
        except Exception as e:
            result["error"] = f"Unexpected error: {str(e)}"
        finally:
            # Step 7: Cleanup temporary directory
            if temp_dir and os.path.exists(temp_dir):
                progress.update(task, description="Cleaning up temporary files...")
                shutil.rmtree(temp_dir)
                progress.advance(task)

    # Update success condition to ensure no stderr or unexpected errors
    result["success"] = result["success"] and not result["error"] and not result["stderr"]
    return result

def format_tool_output(execution_result: Dict[str, Union[str, bool]]) -> str:
    if execution_result["success"]:
        return f"Tool executed successfully. Output:\n{execution_result['stdout']}\n"
    else:
        return (
            f"Tool execution failed.\n"
            f"Error: {execution_result['error'] if execution_result['error'] else 'No additional error details.'}\n"
            f"Standard Error: {execution_result['stderr']}\n"
            f"Standard Output: {execution_result['stdout']}\n"
            f"Script:\n{execution_result['script']}\n"
            f"Dependencies:\n{execution_result['pip_install']}\n"
        )


In [6]:
from typing import Optional, Union

from pydantic import BaseModel, Field


class Success(BaseModel):
    final_answer: str = Field(..., description="The final result or answer to the task if the execution succeeded.")

class Failure(BaseModel):
    analysis: str = Field(..., description="A detailed explanation of why the tool execution failed.")
    suggestions: str = Field(..., description="Suggestions for fixing the issue or alternative approaches to achieve the task.")

class Result(BaseModel):
    success: bool = Field(..., description="Indicates if the tool execution was successful.")
    result: Union[Success, Failure] = Field(..., description="The result of the analysis, including success or failure details.")
    
@brain_client.reasoner(schema=Result)
def result_analyzer(tool_output: str, task: str):
    """
    Analyzes script execution results focusing on zero human intervention requirement.
    """
    system_prompt = """Analyze Python script results for true autonomy - script must run without any human intervention.

Success case:
Important : If tool output does not have any errors and tool has executed successfully, provide the final answer in a human-readable format.
Successful tool use might have no output as it can be saving a file or updating a database etc.
Only go to error cases if there is an error in the tool output.
- Mention success as True
Extract meaningful result from tool_output and present as:
- Direct answer addressing the original task
- Natural, human-readable format
- No technical details unless relevant
- No mention of script/code aspects


Failure case - Check for:
1. Manual Requirements (like these, but it can be more):
   - API keys/credentials needed
   - User prompts/inputs
   - Configuration steps
   - Runtime user decisions
   - If this task cannot be done without mocking or simulating or manual intervention please mention clearly this as a main reason for failure and ask the tool to reply saying this cannot be done

Suggestions:
1. If failure due to manual steps, suggest truly autonomous alternatives
2. if the script uses API or mock data or synthetic data, recomend it to come up with a solution that can be done without these.
3. ensure the script is fully autonomous and can be run without any manual intervention or need for external api keys or manual inputs.
4. Ensure there are no Mock or synthetic data used in the script.

2. Script should:
   - Use standard library or pip packages
   - Make API calls if no credentials needed
   - Handle all data programmatically
   - Run start-to-finish autonomously
   - Should not mock or simulate any data or use placeholders
   - Should make sure we are achiving the task as expected without artificial hallucinated data.
   - You have access to internet and you can download any data from internet if needed.

Key principle:
If task requires ANY manual steps (API keys, inputs, etc):
- Clearly state why it can't be autonomous
- Don't suggest mock/placeholder solutions
- Only suggest truly automatic alternatives"""

    user_prompt = f"""Analyze result for: "{task}"
Output: "{tool_output}"

If tool output does not have any errors and tool has executed successfully, provide the final answer in a human-readable format.

Success: Direct task response
Failure: Explain manual intervention needs"""

    return system_prompt, user_prompt
result_analyzer_id = result_analyzer.register()

In [7]:
from typing import List, Optional

from pydantic import BaseModel, Field


class ToolExecutionResult(BaseModel):
    """
    Encapsulates the result of the autonomous tool's execution process.
    """
    final_result: Optional[str] = Field(None, description="The final result or answer to the task if successful.")
    final_script: Optional[str] = Field(None, description="The final generated Python script.")
    pip_dependencies: List[str] = Field(..., description="The list of pip dependencies required for the script.")
    execution_output: Optional[str] = Field(None, description="The output from executing the script.")
    success: bool = Field(..., description="Indicates if the task was successfully executed.")
    failure_reason: Optional[str] = Field(None, description="The reason for failure if the task could not be completed.")


@brain_client.multi_agent
def autonomous_tool(task: str, max_trials: int = 5, verbose: bool = True) -> ToolExecutionResult:
    """
    Multi-agent workflow to iteratively generate, execute, and analyze tools for a given task.

    Args:
        task (str): The task description.
        max_trials (int): Maximum number of attempts to generate and execute a successful tool.
        verbose (bool): Whether to print detailed logs.

    Returns:
        ToolExecutionResult: Structured result of the autonomous tool's execution.
    """
    for iteration in range(max_trials):
        print(f"{'=' * 10} Iteration {iteration + 1} {'=' * 10}")

        # Add context for retries if not the first iteration
        if iteration != 0:
            task_context = (
                f"Last tool failed because {tool_output.completed}. "
                f"Last tool script:\n{tool_output.script if tool_output.completed is True else 'N/A'}\n"
                f"Pip dependencies: {tool_output.pip_install if tool_output.completed is True else 'N/A'}\n"
            )
        else:
            task_context = ""

        # Step 1: Generate the tool
        tool_output = brain_client.use(autonomous_tool_id)(task=task_context + task)

        # If tool generation failed, return failure details
        if tool_output.completed is not True:
            if verbose:
                print(f"Tool Generation Failed: {tool_output.completed}")
            return ToolExecutionResult(
                final_result=None,
                final_script=None,
                pip_dependencies=[],
                execution_output=None,
                success=False,
                failure_reason=str(tool_output.completed)
            )

        # Step 2: Execute the generated script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)

        # Step 3: Format execution result for analysis
        context = format_tool_output(exec_output)

        # Step 4: Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)

        # Verbose logging for debugging
        if verbose:
            print(f"Result Analysis: {res.result}")

        # If successful, return the result details
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            return ToolExecutionResult(
                final_result=res.result.final_answer,
                final_script=tool_output.script,
                pip_dependencies=tool_output.pip_install,
                execution_output=context,
                success=True
            )

    # If all attempts fail, return the failure details
    return ToolExecutionResult(
        final_result=None,
        final_script=tool_output.script if tool_output.completed is True else None,
        pip_dependencies=tool_output.pip_install if tool_output.completed is True else [],
        execution_output=None,
        success=False,
        failure_reason=res.result.analysis if 'res' in locals() and not res.success else "Unknown failure."
    )

In [ ]:
task="What is the temperature in Downtown Toronto today?"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: analysis='The execution failed because the script relies on simulated data rather than fetching 
real-time data from an API. Real-time temperature data for Downtown Toronto cannot be obtained without API keys or 
credentials, making the task unachievable without manual intervention. The task cannot be performed autonomously.' 
suggestions='To achieve true autonomy, a new approach should involve using a public weather API that does not 
require authentication or credentials. The script should be modified to directly call such an API and handle the 
data programmatically, ensuring it can run without the need for any simulated or mocked data.'

========== Iteration 2 ==========

Tool Generation Failed: False

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='False'

In [ ]:
task="search for top 3 articles on autonomous driving and give back the titles"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: analysis="The execution of the script failed to retrieve the top 3 articles on autonomous driving 
due to its reliance on an external web scraping approach that requires a stable internet connection and access to 
Google's search results page without any required API keys or manual input. Additionally, web scraping can often be
blocked or limited by the target websites, making it an unreliable method for autonomous tasks." suggestions='To 
achieve truly autonomous results without any manual intervention, consider using a reliable news API (such as 
NewsAPI or Bing News Search API) that does not require manual input or credentials for basic usage. This way, you 
can programmatically retrieve the latest articles on autonomous driving while ensuring full autonomy in the 
execution.'

========== Iteration 2 ==========

Tool Generation Failed: True

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='True'

In [ ]:
task="calculate the sum of numbers from 1 to 100_0000_000"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The sum of numbers from 1 to 100000000 is: 5000000050000000'

Final Tool Output:
Tool executed successfully. Output:
The sum of numbers from 1 to 100000000 is: 5000000050000000

Final Result: final_result='The sum of numbers from 1 to 100000000 is: 5000000050000000' final_script="def 
sum_of_numbers(n):\n    return sum(range(1, n + 1))\n\nif __name__ == '__main__':\n    try:\n        n = 
100_000_000  # This is the upper limit as requested\n        result = sum_of_numbers(n)\n        print(f'The sum of
numbers from 1 to {n} is: {result}')\n    except Exception as e:\n        print(f'An error occurred: {e}')" 
pip_dependencies=[] execution_output='Tool executed successfully. Output:\nThe sum of numbers from 1 to 100000000 
is: 5000000050000000\n' success=True failure_reason=None

In [ ]:
task="Multiply 777_82 times 67625_7762_8"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The multiplication of 77782 and 6762577628 results in 526006813061096.'

Final Tool Output:
Tool executed successfully. Output:
The result of multiplying 77782 and 6762577628 is: 526006813061096

Final Result: final_result='The multiplication of 77782 and 6762577628 results in 526006813061096.' 
final_script='def multiply_numbers(a, b):\n    """Multiply two numbers and return the result."""\n    return a * 
b\n\n# Test Data\nnum1 = 77782\nnum2 = 6762577628\n\ntry:\n    result = multiply_numbers(num1, num2)\n    
print(f\'The result of multiplying {num1} and {num2} is: {result}\')\nexcept Exception as e:\n    print(f\'An error
occurred: {e}\')' pip_dependencies=[] execution_output='Tool executed successfully. Output:\nThe result of 
multiplying 77782 and 6762577628 is: 526006813061096\n' success=True failure_reason=None

In [ ]:
print(777_82*67625_7762_8)

526006813061096

In [ ]:
task="Get ellon musk's latest tweet"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Tool Generation Failed: The task cannot be completed because accessing Elon Musk's latest tweet requires connecting
to the Twitter API, which necessitates using an API key and other authentication methods that fall outside the 
constraints provided. Therefore, it cannot be done without external dependencies or secret configurations.

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason="The task cannot be completed because accessing Elon Musk's latest tweet requires connecting to the 
Twitter API, which necessitates using an API key and other authentication methods that fall outside the constraints
provided. Therefore, it cannot be done without external dependencies or secret configurations."

## Tool planner

In [8]:
from typing import List

from pydantic import BaseModel, Field


class Instructions(BaseModel):
    feasibility: str = Field(..., description="Analysis of whether the task can be completed autonomously, including any potential blockers")
    implementation_plan: str = Field(..., description="Detailed step-by-step plan for implementing the solution, including function breakdowns and data handling approaches")
    requirements: List[str] = Field(..., description="List of specific requirements including Python packages, error handling needs, and data processing requirements")
    execution_flow: str = Field(..., description="Precise description of how the script should execute, including data handling, function calls, and expected outcomes")
    is_autonomous: bool = Field(..., description="Whether the task can be completed without any human intervention, API keys, or manual human dependencies")

    

@brain_client.reasoner(schema=Instructions)
def feature_planner(task: str):
    """
    Generates detailed instructions for building autonomous Python scripts.
    """
    system_prompt = """You are a Python architecture planner specialized in designing autonomous scripts. Your role is to analyze tasks and provide clear, actionable instructions that ensure zero human intervention.

Analyze each task for true autonomy by considering:

Autonomy Requirements:
- Script must run without any API keys or credentials
- No user inputs can be required
- No environment variables or configuration needed
- No external service dependencies
- No system-specific resources

Input guidelines:
- make sure you specify what exact inputs are needed for the script to do the given task.

Implementation Guidelines:
- Prefer Python standard library solutions
- Use only common, stable PyPI packages if needed
- Include comprehensive error handling
- No mock or simulated data allowed
- No placeholder values
- Try to be Clever to come up with a solution that can be done without any blockers like apikey or manual input.
- We can have any pip packages that is needed and we can install it on the fly
- You have access to internet and you can download any data from internet if needed.

Data Handling Rules:
- If task mentions file paths (e.g., "./something.something"), provide exact handling instructions
- Data must be programmatically generated or bundled
- No external data sources allowed if we cannot access them and needs api key or manual intervention
- No simulated or mock data

Your output should:
1. Assess if task can be truly autonomous
2. Provide detailed implementation steps
3. Specify exact execution flow
4. List concrete requirements
5. Flag any autonomy blockers

If the task cannot be autonomous:
- Set is_autonomous to false
- Explain specific blockers
- Do not suggest mock data workarounds
- Clearly state why task requires human intervention

Important: 
If any feedback is provided, make sure write your answer based on that feedback. Based on feedback update required packages or implementation plan etc..
If you think previous attempt failed because of some specific reason, make sure to either  address that in your answer or come up with a completely new solution based on the feedback provided to sovle the task exactly.
"""

    user_prompt = f"""Analyze this task and provide implementation instructions: "{task}"

Determine:
1. Is this task possible without any human intervention?
2. What exact implementation steps are needed?
3. How should data and paths be handled?
4. What specific requirements must be met?

Return Instructions with:
- Complete feasibility analysis
- Detailed implementation plan
- Specific requirements
- Precise execution flow
- Autonomy determination.
- Make sure to give detailed unambigious plan.

You Can run python scripts and install any pip packages without human intervention

Important: If any feedback is provided, make sure write your answer based on that feedback. Based on feedback update required packages or implementation plan etc.."""

    return system_prompt, user_prompt
feature_planner_id = feature_planner.register()

In [9]:
@brain_client.reasoner(schema=Tool)
def autonomous_planner_tool(instructions: Instructions):
    """
    Generates standalone Python scripts based on detailed implementation instructions.
    """
    system_prompt = """You are an expert Python script generator that creates fully autonomous code based on provided implementation instructions. Your goal is to generate scripts that run without any human intervention.
    
You Can run python scripts and install any pip packages without human intervention

Core Requirements:
1. Script Generation Rules:
   - No external inputs (API keys, credentials, user prompts)
   - No mock or simulated data
   - No manual configuration needs
   - No environment variables
   - No system-specific resources
   - No external service dependencies requiring authentication

2. Development Guidelines:
   - Prioritize Python standard library solutions
   - Use only widely-adopted PyPI packages when necessary
   - Implement comprehensive error handling
   - Generate data programmatically if needed
   - Handle all edge cases gracefully
   - Follow the provided implementation plan exactly
   - Use exact paths if specified in the implementation plan
   - We can have any pip packages that is needed and we can install it on the fly
   - You have access to internet and you can download any data from internet if needed.

3. Validation Requirements:
   - Ensure script runs without any user intervention
   - Verify no external dependencies are needed
   - Confirm no mock/simulated data is used
   - Follow execution flow from instructions
   - Implement all specified error handling

If task cannot be completed autonomously:
- Set completed to detailed explanation
- Do not attempt partial implementations
- Do not suggest mock data alternatives
- Explain specific blockers"""

    user_prompt = f"""Generate a Python script following these instructions:

Feasibility: {instructions.feasibility}
Implementation Plan: {instructions.implementation_plan}
Requirements: {instructions.requirements}
Execution Flow: {instructions.execution_flow}
Is Autonomous: {instructions.is_autonomous}

Return Tool schema with:
- Complete, autonomous Python script
- Required pip packages. Make sure to give all the packages that is needed to run the script, including auxiliary packages.
- Success status or detailed explanation of blockers"""

    return system_prompt, user_prompt
autonomous_planner_tool_id = autonomous_planner_tool.register()

In [10]:
from typing import List, Optional

from pydantic import BaseModel, Field


class ToolExecutionResult(BaseModel):
    """
    Encapsulates the result of the autonomous tool's execution process.
    """
    final_result: Optional[str] = Field(None, description="The final result or answer to the task if successful.")
    final_script: Optional[str] = Field(None, description="The final generated Python script.")
    pip_dependencies: List[str] = Field(..., description="The list of pip dependencies required for the script.")
    execution_output: Optional[str] = Field(None, description="The output from executing the script.")
    success: bool = Field(..., description="Indicates if the task was successfully executed.")
    failure_reason: Optional[str] = Field(None, description="The reason for failure if the task could not be completed.")

@brain_client.multi_agent
def autonomous_planner_tool(task: str, max_trials: int = 5, verbose: bool = True) -> ToolExecutionResult:
    """
    Multi-agent workflow to plan, generate, execute, and analyze tools for a given task.
    """
    for iteration in range(max_trials):
        if verbose:
            print(f"{'=' * 10} Iteration {iteration + 1} {'=' * 10}")

        # Step 1: Generate implementation plan
        plan = brain_client.use(feature_planner_id)(task=task)
        
        # If task is not autonomous based on plan, return failure
        if not plan.is_autonomous:
            return ToolExecutionResult(
                final_result=None,
                final_script=None,
                pip_dependencies=[],
                execution_output=None,
                success=False,
                failure_reason=plan.feasibility
            )

        # Step 2: Generate tool based on plan
        tool_output = brain_client.use(autonomous_planner_tool_id)(instructions=plan)

        # If tool generation failed, continue to next iteration
        if tool_output.completed is not True:
            if verbose:
                print(f"Tool Generation Failed: {tool_output.completed}")
            continue

        # Step 3: Execute the generated script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)

        # Step 4: Format execution result for analysis
        context = format_tool_output(exec_output)

        # Step 5: Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)

        if verbose:
            print(f"Result Analysis: {res.result}")

        # If successful, return the result details
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            return ToolExecutionResult(
                final_result=res.result.final_answer,
                final_script=tool_output.script,
                pip_dependencies=tool_output.pip_install,
                execution_output=context,
                success=True
            )

        # If failed, add context for next iteration
        task = (
            f"Previous Implementation plan:\n{plan.implementation_plan}"
            f"Previous attempt failed.\nAnalysis: {res.result.analysis}\n"
            f"Suggestions: {res.result.suggestions}\n"
            f"Original task: {task}\n"
        )

    # If all attempts fail, return the failure details
    return ToolExecutionResult(
        final_result=None,
        final_script=tool_output.script if tool_output.completed is True else None,
        pip_dependencies=tool_output.pip_install if tool_output.completed is True else [],
        execution_output=None,
        success=False,
        failure_reason=res.result.analysis if 'res' in locals() and not res.success else "Maximum iterations reached without success."
    )

In [ ]:
task="Extract the top 5 most frequent words from the text file ./input.txt and ignore common stopwords."
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The task was executed successfully, and the top 5 most frequent words found in the 
text file are:\n\n1. may: 1664\n2. _: 1064\n3. acid: 734\n4. death: 728\n5. cases: 549\n\nThis analysis effectively
disregards common stopwords as required.'

Final Tool Output:
Tool executed successfully. Output:
Top 5 words:
may: 1664
_: 1064
acid: 734
death: 728
cases: 549

Final Result: final_result='The task was executed successfully, and the top 5 most frequent words found in the text
file are:\n\n1. may: 1664\n2. _: 1064\n3. acid: 734\n4. death: 728\n5. cases: 549\n\nThis analysis effectively 
disregards common stopwords as required.' final_script='import collections\nimport re\nimport os\n\n# Function to 
load the text file\n\ndef load_file(file_path):\n    try:\n        if not os.path.isfile(file_path):\n            
raise FileNotFoundError(f"File not found: {file_path}")\n        with open(file_path, \'r\', encoding=\'utf-8\') as
file:\n            content = file.read()\n        return content\n    except Exception as e:\n        return 
str(e)\n\n# List of common stopwords\nstopwords = {\n    \'i\', \'me\', \'my\', \'myself\', \'we\', \'our\', 
\'ours\', \'ourselves\',\n    \'you\', \'your\', \'yours\', \'yourself\', \'yourselves\', \'he\', \'him\',\n    
\'his\', \'himself\', \'she\', \'her\', \'hers\', \'herself\', \'it\', \'its\',\n    \'itself\', \'they\', 
\'them\', \'their\', \'theirs\', \'themselves\', \'what\',\n    \'which\', \'who\', \'whom\', \'this\', \'that\', 
\'these\', \'those\', \'am\', \'is\', \'are\',\n    \'was\', \'were\', \'be\', \'been\', \'being\', \'have\', 
\'has\', \'had\', \'having\',\n    \'do\', \'does\', \'did\', \'doing\', \'a\', \'an\', \'the\', \'and\', \'but\', 
\'if\',\n    \'or\', \'because\', \'as\', \'until\', \'while\', \'of\', \'at\', \'by\', \'for\',\n    \'with\', 
\'about\', \'against\', \'between\', \'into\', \'through\', \'during\',\n    \'before\', \'after\', \'above\', 
\'below\', \'to\', \'from\', \'up\', \'down\',\n    \'in\', \'out\', \'on\', \'off\', \'over\', \'under\', 
\'again\', \'further\',\n    \'then\', \'once\', \'here\', \'there\', \'when\', \'where\', \'why\', \'how\',\n    
\'all\', \'any\', \'both\', \'each\', \'few\', \'more\', \'most\', \'other\',\n    \'some\', \'such\', \'no\', 
\'nor\', \'not\', \'only\', \'own\', \'same\',\n    \'so\', \'than\', \'too\', \'very\', \'s\', \'t\', \'can\', 
\'will\', \'just\',\n    \'don\', \'should\', \'now\'\n}\n\n# Function to process the text\n\ndef 
process_text(content):\n    # Tokenize and clean the text\n    words = re.findall(r\'\\b\\w+\\b\', 
content.lower())\n    # Filter out stopwords\n    filtered_words = \n    # Count frequencies\n    word_counts = 
collections.Counter(filtered_words)\n    return word_counts\n\n# Function to extract the top 5 words\n\ndef 
get_top_words(word_counts, num=5):\n    return word_counts.most_common(num)\n\n# Main execution flow\nif __name__ 
== \'__main__\':\n    file_path = \'./input.txt\'\n    content = load_file(file_path)\n    if isinstance(content, 
str) and content.startswith(\'File not found\'):\n        print(content)\n    else:\n        word_counts = 
process_text(content)\n        if not word_counts:\n            print("No valid words found.")\n        else:\n    
top_words = get_top_words(word_counts)\n            print("Top 5 words:")\n            for word, count in 
top_words:\n                print(f\'{word}: {count}\')' pip_dependencies=[] execution_output='Tool executed 
successfully. Output:\nTop 5 words:\nmay: 1664\n_: 1064\nacid: 734\ndeath: 728\ncases: 549\n' success=True 
failure_reason=None

In [69]:
# get absolute path of a file
file_path="./input.txt"
task=f"use plotly and plot the distribution of word length in the text file {file_path} and fit a nice curve to the distribution and save the plot as file."
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: analysis='The tool execution failed because it could not install the necessary Python packages due
to an issue with the command execution, which may indicate a lack of permissions or an environment problem. As a 
result, the script cannot perform its intended task without the required dependencies.' suggestions='This task 
cannot be completed as described because it relies on installing third-party packages like Pandas, NumPy, and 
Plotly, which requires manual intervention in managing the Python environment. To achieve true autonomy, consider 
using a self-contained tool or package that does not require external dependencies, or ensure the environment is 
set up correctly to allow for the installation of these packages.'

========== Iteration 2 ==========

Result Analysis: analysis='The tool execution failed due to the inability to install necessary Python packages like
pandas, numpy, and plotly. This indicates a lack of permission or an environmental issue, preventing the script 
from running successfully. As a result, without these dependencies, the script cannot complete its intended task of
analyzing the text file and generating the required statistics and plots.' suggestions='To ensure true autonomy in 
this task, it is recommended to set up a proper Python environment where the required packages can be installed 
without issues. Consider using a virtual environment or a containerized solution such as Docker to manage 
dependencies. Additionally, look for alternatives that do not require external libraries, or investigate whether 
the environment can allow for the installation of necessary packages automatically.'

========== Iteration 3 ==========

Result Analysis: final_answer='The histogram showing the distribution of word lengths from the text file has been 
successfully created and saved as "word_length_distribution.png".'

Final Tool Output:
Tool executed successfully. Output:
Success: The histogram has been created and saved as word_length_distribution.png.

Final Result: final_result='The histogram showing the distribution of word lengths from the text file has been 
successfully created and saved as "word_length_distribution.png".' final_script="import pandas as pd\nimport numpy 
as np\nimport plotly.graph_objs as go\nimport os\n\n# Check for the input file existence\ninput_file = 
'./input.txt'\nif not os.path.exists(input_file):\n    print('Error: The file ./input.txt does not exist.')\n    
raise FileNotFoundError('The file ./input.txt does not exist.')\n\ntry:\n    # Read the content of the input file\n
with open(input_file, 'r') as file:\n        text = file.read()\nexcept Exception as e:\n    print(f'Error reading 
the file: {str(e)}')\n    raise e\n\n# Split the text into words and calculate word lengths\nword_lengths = \n\n# 
Create a DataFrame from the word lengths\nword_length_df = pd.DataFrame(word_lengths, columns=['Length'])\n\n# Fit 
a distribution curve using numpy for the histogram data\ntry:\n    mu, sigma = np.mean(word_lengths), 
np.std(word_lengths)\n    x_values = np.linspace(min(word_lengths), max(word_lengths), 100)\n    p = (1/(sigma * 
np.sqrt(2 * np.pi))) * np.exp(- (x_values - mu)**2 / (2 * sigma**2))\nexcept Exception as e:\n    print(f'Error 
fitting distribution: {str(e)}')\n    raise e\n\n# Create a histogram and overlay the fitted curve using 
Plotly\ntry:\n    histogram = go.Figure()\n    histogram.add_trace(go.Histogram(x=word_length_df['Length'], 
name='Word Lengths', opacity=0.75))\n    histogram.add_trace(go.Scatter(x=x_values, y=p * len(word_lengths) * 
(max(word_lengths) - min(word_lengths)) / 100, name='Fitted Curve', mode='lines'))\n    
histogram.update_layout(title='Word Length Distribution', xaxis_title='Length', yaxis_title='Count')\n\n    # Save 
the plot to a PNG file\n    histogram.write_image('word_length_distribution.png')\nexcept Exception as e:\n    
print(f'Error creating histogram or saving the file: {str(e)}')\n    raise e\n\nprint('Success: The histogram has 
been created and saved as word_length_distribution.png.')" pip_dependencies=['pandas', 'numpy', 'plotly', 
'kaleido'] execution_output='Tool executed successfully. Output:\nSuccess: The histogram has been created and saved
as word_length_distribution.png.\n' success=True failure_reason=None

In [73]:
# get absolute path of a file
file_path="./input.txt"
task=f"Identify the most important words in the document using machine learning in the text file {file_path}"
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: analysis="The script execution failed because it requires a manual step to download the necessary 
NLTK resources. Specifically, the `punkt` tokenizer package must be downloaded for the script to function 
correctly, and this download process requires internet access and can't be automated within the script without user
intervention." suggestions="To achieve true autonomy, consider using a pre-built tokenizer that doesn't require 
external downloads, or ensure that all necessary resources are bundled within the script or the environment it runs
in. Another approach could involve using a different text processing library that doesn't need additional 
datasets."

========== Iteration 2 ==========

Result Analysis: analysis="The script execution failed because it requires the manual download of the necessary 
SpaCy model 'en_core_web_sm'. This model is not bundled with the script and cannot be downloaded without user 
intervention. Additionally, the original implementation using NLTK also failed due to the need to download 
resources for tokenization. Thus, the task cannot be performed without manual steps to set up the required models."
suggestions='To achieve true autonomy, consider using a pre-installed tokenizer or a library that comes with 
built-in models compatible with the script, eliminating the need for manual downloads. Alternatively, explore using
simpler text processing libraries that do not require additional model installations.'

========== Iteration 3 ==========

Result Analysis: final_answer='The top important words from the document have been successfully identified and 
saved to the output file.'

Final Tool Output:
Tool executed successfully. Output:
Successfully written top terms to ./output.txt

Final Result: final_result='The top important words from the document have been successfully identified and saved 
to the output file.' final_script="import os\nfrom textblob import TextBlob\nfrom sklearn.feature_extraction.text 
import TfidfVectorizer\n\n# Constants\nINPUT_FILE_PATH = './input.txt'\nOUTPUT_FILE_PATH = './output.txt'\nTOP_N = 
10\n\ndef read_input_file(file_path):\n    try:\n        with open(file_path, 'r') as file:\n            content = 
file.read().strip()\n            if not content:\n                raise ValueError('Input file is empty.')\n       
return content\n    except FileNotFoundError:\n        raise FileNotFoundError(f'Input file not found: 
{file_path}')\n    except PermissionError:\n        raise PermissionError(f'Permission denied to read file: 
{file_path}')\n\n\ndef tokenize_and_clean(text):\n    # Convert text to TextBlob object\n    blob = 
TextBlob(text)\n    # Tokenization and cleaning: lowercase and remove punctuation\n    tokens = \n    return 
tokens\n\n\ndef calculate_tfidf(tokens):\n    vectorizer = TfidfVectorizer(stop_words='english')\n    # Join tokens
as a single document string for TF-IDF processing\n    X = vectorizer.fit_transform([' '.join(tokens)])\n    
tfidf_scores = dict(zip(vectorizer.get_feature_names_out(), X.toarray()[0]))\n    return tfidf_scores\n\n\ndef 
get_top_n_terms(tfidf_scores, n):\n    sorted_terms = sorted(tfidf_scores.items(), key=lambda x: x[1], 
reverse=True)\n    return sorted_terms[:n]\n\n\ndef write_output_file(file_path, top_terms):\n    try:\n        
with open(file_path, 'w') as file:\n            for term, score in top_terms:\n                file.write(f'{term}:
{score}\\n')\n    except PermissionError:\n        raise PermissionError(f'Permission denied to write file: 
{file_path}')\n    except Exception as e:\n        raise Exception(f'An error occurred while writing to 
{file_path}: {str(e)}')\n\n\ndef main():\n    try:\n        # Step 2: Read the input file\n        text = 
read_input_file(INPUT_FILE_PATH)\n        # Step 3: Tokenization and cleaning\n        tokens = 
tokenize_and_clean(text)\n        # Step 4: TF-IDF calculation\n        tfidf_scores = calculate_tfidf(tokens)\n   
# Step 5: Get top N terms\n        top_terms = get_top_n_terms(tfidf_scores, TOP_N)\n        # Step 6: Write output
file\n        write_output_file(OUTPUT_FILE_PATH, top_terms)\n        print(f'Successfully written top terms to 
{OUTPUT_FILE_PATH}')\n    except Exception as e:\n        print(f'Error: {str(e)}')\n\n\nif __name__ == 
'__main__':\n    main()" pip_dependencies=['textblob', 'scikit-learn'] execution_output='Tool executed 
successfully. Output:\nSuccessfully written top terms to ./output.txt\n' success=True failure_reason=None

In [90]:
task="search for top 3 articles on autonomous driving in google and give back the titles"
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The script for achieving true autonomy has executed successfully and without any 
errors. The operation completed as intended.'

Final Tool Output:
Tool executed successfully. Output:
Top 3 article titles:
- What Are Self-Driving Cars? The Technology Explained - Forbes
- Autonomous vehicles: theoretical and practical challenges
- The evolving safety and policy challenges of self-driving cars

Final Result: final_result='The script for achieving true autonomy has executed successfully and without any 
errors. The operation completed as intended.' final_script="import requests\nfrom bs4 import BeautifulSoup\n\n# 
Function to perform Google search and extract article titles\n\ndef fetch_article_titles(query):\n    try:\n       
# Perform a GET request to Google Search\n        headers = {'User-Agent': 'Mozilla/5.0'}\n        url = 
'https://www.google.com/search?q=' + query\n        response = requests.get(url, headers=headers)\n        
response.raise_for_status()  # Raise an error for bad responses\n\n        # Parse the HTML content using 
BeautifulSoup\n        soup = BeautifulSoup(response.text, 'html.parser')\n        titles = []\n\n        # Find 
all search result titles\n        for item in soup.find_all('h3'):  # Google's titles usually are in <h3> tags\n   
titles.append(item.get_text())\n            if len(titles) >= 3:\n                break\n\n        return 
titles\n\n    except requests.exceptions.RequestException as e:\n        print(f'Error during requests to Google: 
{e}')\n    except Exception as e:\n        print(f'An error occurred: {e}')\n    return []\n\n# Main execution 
flow\nif __name__ == '__main__':\n    query = 'autonomous driving articles'\n    article_titles = 
fetch_article_titles(query)\n    if article_titles:\n        print('Top 3 article titles:')\n        for title in 
article_titles:\n            print(f'- {title}')\n    else:\n        print('No articles found or an error occurred 
while fetching articles.')" pip_dependencies=['requests', 'beautifulsoup4'] execution_output='Tool executed 
successfully. Output:\nTop 3 article titles:\n- What Are Self-Driving Cars? The Technology Explained - Forbes\n- 
Autonomous vehicles: theoretical and practical challenges\n- The evolving safety and policy challenges of 
self-driving cars\n' success=True failure_reason=None

In [11]:
file_path="./input.txt"
task=f"Identify words with unusually high or low frequency outliers in the text file {file_path} using sklearn, and plot and save a 2d outlier plot using plotly"
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Output()

Result Analysis: final_answer='The task of identifying words with unusually high or low frequency outliers in the 
specified text file was completed successfully. The results have been processed, and a 2D outlier plot has been 
generated and saved using Plotly. No further actions are required.'

Final Tool Output:
Tool executed successfully. Output:

Final Result: final_result='The task of identifying words with unusually high or low frequency outliers in the 
specified text file was completed successfully. The results have been processed, and a 2D outlier plot has been 
generated and saved using Plotly. No further actions are required.' final_script='import pandas as pd\nfrom 
sklearn.feature_extraction.text import CountVectorizer\nfrom sklearn.ensemble import IsolationForest\nimport 
plotly.express as px\nimport os\n\n# Step 2: Read text data from \'./input.txt\'\ntry:\n    with 
open(\'./input.txt\', \'r\', encoding=\'utf-8\') as file:\n        text_data = file.read()\nexcept 
FileNotFoundError:\n    raise FileNotFoundError("The file \'./input.txt\' was not found.")\nexcept Exception as 
e:\n    raise Exception(f"An error occurred while reading the file: {str(e)}")\n\n# Step 3: Use CountVectorizer to 
transform text data into a frequency matrix\ntry:\n    vectorizer = CountVectorizer()  \n    freq_matrix = 
vectorizer.fit_transform()\nexcept Exception as e:\n    raise Exception(f"An error occurred during vectorization: 
{str(e)}")\n\n# Step 4: Calculate word frequencies\ntry:\n    word_freq = freq_matrix.toarray().flatten()\n    
words = vectorizer.get_feature_names_out()\n    freq_df = pd.DataFrame({\'Word\': words, \'Frequency\': 
word_freq})\nexcept Exception as e:\n    raise Exception(f"An error occurred while calculating word frequencies: 
{str(e)}")\n\n# Step 5: Implement Isolation Forest\ntry:\n    isolation_forest = 
IsolationForest(contamination=0.1)\n    isolation_forest.fit(freq_df[[\'Frequency\']])\nexcept Exception as e:\n   
raise Exception(f"An error occurred while fitting the Isolation Forest model: {str(e)}")\n\n# Step 6: Extract the 
outliers\ntry:\n    freq_df[\'Outlier\'] = isolation_forest.predict(freq_df[[\'Frequency\']])\n    outliers_df = 
freq_df[freq_df[\'Outlier\'] == -1]\nexcept Exception as e:\n    raise Exception(f"An error occurred while 
extracting outliers: {str(e)}")\n\n# Step 7: Create a 2D scatter plot using Plotly\ntry:\n    fig = 
px.scatter(freq_df, x=\'Word\', y=\'Frequency\', title=\'Word Frequencies with Outliers\',\n                     
color=(freq_df[\'Outlier\'] == -1).astype(str),\n                     labels={\'color\': \'Outlier\'})\nexcept 
Exception as e:\n    raise Exception(f"An error occurred while creating the plot: {str(e)}")\n\n# Step 8: Save the 
plot as a PNG file\ntry:\n    fig.write_image(\'./output_plot.png\')\nexcept Exception as e:\n    raise 
Exception(f"An error occurred while saving the plot: {str(e)}")\n' pip_dependencies=['pandas', 'scikit-learn', 
'plotly', 'kaleido'] execution_output='Tool executed successfully. Output:\n\n' success=True failure_reason=None